In [1]:
import numpy as np
import scipy.stats as st
import scipy.optimize as opt

In [2]:
def prob_1(f, a, b, N, method):
    interval = (b-a)/N
    func = np.vectorize(f)
    x_1 = np.sum(func(np.linspace(a+(0.5*interval),b-(0.5*interval),N)))
    x_2 = np.sum(func(np.linspace(a+interval,b-interval,N-1)))
    if method=='midpoint':
        integral = interval*x_1
    elif method=='trapezoid':
        integral = 0.5*interval*(f(a)+f(b)+2*x_2)
    elif method=='Simpsons':
        integral = (1/6)*interval*(f(a)+f(b)+4*x_1+2*x_2)
    else:
        raise ValueError("Undefined method!")
    return integral
g = lambda x: 0.1*x**4-1.5*x**3+0.53*x**2+2*x+1
N = 1000
A = 4373+(1/3)
print("Midpoint Difference:", prob_1(g, -10, 10, N, 'midpoint')-A)
print("Trapezoid Difference:", prob_1(g, -10, 10, N, 'trapezoid')-A)
print("Simpsons Difference:", prob_1(g, -10, 10, N, 'Simpsons')-A)

Midpoint Difference: -0.0136866573321
Trapezoid Difference: 0.0273733226686
Simpsons Difference: 2.668457455e-09


In [6]:
def prob_2(mu, sigma, N, k):
    a = mu -  sigma*k
    b = mu + sigma*k
    z = np.linspace(a,b,N)
    w = np.zeros(N)
    w[0] = st.norm.cdf(0.5*(z[0]+z[1]),loc=mu,scale=sigma)
    w[N-1] = 1-st.norm.cdf(0.5*(z[N-2]+z[N-1]),loc=mu,scale=sigma)
    for i in range(1,N-1):
        zmin = (z[i-1]+z[i])*0.5
        zmax = (z[i+1]+z[i])*0.5
        f = lambda x: st.norm.pdf(z[i],loc=mu,scale=sigma)
        w[i] = prob_1(f, zmin, zmax, N, 'Simpsons')
    return z,w
prob_2(0, 1, 10, 3)

(array([-3.        , -2.33333333, -1.66666667, -1.        , -0.33333333,
         0.33333333,  1.        ,  1.66666667,  2.33333333,  3.        ]),
 array([ 0.00383038,  0.01748126,  0.06631809,  0.16131382,  0.25158882,
         0.25158882,  0.16131382,  0.06631809,  0.01748126,  0.00383038]))

In [8]:
#porblem 3
def prob_3(mu, sigma, N, k):
    nodes, weights = prob_2(mu, sigma, N, k)
    nodes = np.exp(nodes)
    return nodes, weights
prob_3(0, 1, 10, 3)

(array([  0.04978707,   0.09697197,   0.1888756 ,   0.36787944,
          0.71653131,   1.39561243,   2.71828183,   5.29449005,
         10.3122585 ,  20.08553692]),
 array([ 0.00383038,  0.01748126,  0.06631809,  0.16131382,  0.25158882,
         0.25158882,  0.16131382,  0.06631809,  0.01748126,  0.00383038]))

In [10]:
def prob_4(nodes, weights):
    return np.sum(nodes@weights)
mu = 10.5
sigma = 0.8
N = 50
k = 3
N, W= prob_3(mu, sigma, N, k)
income_expect = prob_4(N, W)
actual_expect = np.exp(mu+(sigma**2)/2)
print(actual_expect)
print(income_expect)

50011.0870085
49862.6503867


In [11]:
a = -10
b = 10

def v_func(x):
    vec = np.empty(6)
    for i in range(6):
        j = i+1
        rhs = 0
        for k in range(3):
            rhs += x[k]*(x[k+3]**i)
        vec[i] = (b**j-a**j)/j - rhs
    return vec

x_0 = np.ones(6)
x = opt.root(v_func,x_0,tol=1e-14).x
integral = 0
for k in range(3):
    integral += x[k]*(g(x[k+3]))
actual = 4373+(1/3) 
print( integral-actual)

-9.09494701773e-13


In [12]:
from scipy.integrate import quad as quad
scipy_estimate = quad(g,a,b)[0]
print( scipy_estimate-actual)

9.094947017729282e-13
